In [1]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm

np.random.seed(323)

In [2]:
def reinforce_function(func, n, *args):
    """ Repeat a function with certain parameters several times and return the
    mean result

    Parameters
    ----------
    func : function
        The function to be repeated
    n : int
        The repeating time
    args : tuple
        The parameters for the function

    Returns
    -------
    mean_value : float
        The mean value of n times running
    """

    value = []
    for i in range(n):
        value.append(func(*args))
    mean_value = np.mean(value, axis=0)
    
    return mean_value


def reinforce_series_function(func, n, *args):
    """ Repeat a function, which generates a series of results, with certain
    parameters several times and return the estimator's value and variance

    Parameters
    ----------
    func : function
        The function to be repeated
    n : int
        The repeating time
    args : tuple
        The parameters for the function

    Returns
    -------
    mean : float
        The mean value of all the results
    variance : float
        The variance of the estimator
    """

    value_list = []
    for i in range(n):
        value_list.append(func(*args))
    value = np.hstack(value_list)
    mean = np.mean(value)
    variance = np.var(value) / value.shape[0]
    
    return mean, variance


# Problem 1

In [3]:
def euro_basket_option(n):
    """

    Parameters
    ----------
    n : int or float
        The simulation size

    Returns
    -------
    v_red : ndarray
        The simulated present values

    """

    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the contract and assets
    r, delta, sigma, rho = 0.05, 0.02, 0.3, 0.2
    s0 = np.full((3, 1), 100)
    k, t = 100, 1
    # Define parameters for the multivariate normal distribution
    mean, cov = np.zeros(3), np.full((3, 3), rho)
    np.fill_diagonal(cov, 1)
    rng = np.random.default_rng()
    # Generate multivariate normal random variables
    x = rng.multivariate_normal(mean, cov, size=n).T
    # Use the antithetic method to reduce the variance
    x = np.hstack((x, -x))
    # Calculate terminal prices
    st = s0 * np.exp((r - delta - sigma ** 2 / 2) * t + sigma * np.sqrt(t) * x)
    # Return present values of the contract
    v = np.exp(-r * t) * np.maximum(st[0, :] + st[1, :] - st[2, :] - k, 0)
    v_red = (v[:n] + v[n:]) / 2

    return v_red



In [4]:
# Output the results
euro_basket_price, euro_basket_var = reinforce_series_function(
    euro_basket_option, 10, 1e7)
print('European Basket Option:')
print('Price: {:.2f}'.format(euro_basket_price))
print('Variance of the Estimator: {:.4e}'.format(euro_basket_var))

European Basket Option:
Price: 20.40
Variance of the Estimator: 2.8921e-06


In [5]:
def berm_basket_option(n):
    """

    Parameters
    ----------
    n : int or float
        The simulation size

    Returns
    -------
    ev : float
        The present value of the contract
    se2 : float
        The variance of the estimator

    """

    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the contract and assets
    r, delta, sigma, rho = 0.05, 0.02, 0.3, 0.2
    k, t, m = 100, 1, 12
    h = t / m
    s = np.empty(shape=(3, 2 * n, m + 1))
    s[:, :, 0] = 100
    # Define parameters for the multivariate normal distribution
    mean, cov = np.zeros(3), np.full((3, 3), rho)
    np.fill_diagonal(cov, 1)
    rng = np.random.default_rng()
    # Generate multivariate normal random variables
    x = rng.multivariate_normal(mean, cov, size=(m, n)).T
    # Use the antithetic method to reduce the variance
    x = np.hstack((x, -x))
    # Generate price paths for all simulations
    for i in range(m):
        s[:, :, i + 1] = s[:, :, i] * np.exp(
            (r - delta - sigma ** 2 / 2) * h + sigma * np.sqrt(h) * x[:, :, i])
    # Initialize containers
    v = np.empty(shape=(m, 2 * n))
    cv = np.empty(shape=(m - 1, 2 * n))

    # Define a function to calculate the intrinsic value
    def cal_intrin_value(s, k, i):
        return np.maximum(s[0, :, i] + s[1, :, i] - s[2, :, i] - k, 0)

    # Calculate the value of the option at time t = T
    v[m - 1, :] = np.maximum(cal_intrin_value(s, k, m), 0)
    # Backward induction
    for i in range(m - 1, -1, -1):
        # Calculate basis functions' values
        X = np.vstack([s[j, :, i] for j in range(3)] +
                      [s[j, :, i] * s[p, :, i]
                       for j in range(3) for p in range(j, 3)] +
                      [s[j, :, i] * s[p, :, i] * s[q, :, i] for j in range(3)
                       for p in range(j, 3) for q in range(p, 3)] + 
                      [np.max(s[:, :, i], axis=0)] + 
                      [np.maximum(np.max(s[:, :, i], axis=0) - k, 0)])
        X = sm.add_constant(X.T)
        # Create and fit the OLS model
        model = sm.OLS(v[i, :], X)
        results = model.fit()
        # Using the coefficients, calculate the expected value
        cv[i - 1, :] = np.maximum(np.exp(-r * h) * results.fittedvalues, 0)
        # Set the option's value as its discounted value
        v[i - 1, :] = np.exp(-r * h) * v[i, :]
        # If the option's intrinsic value is greater than its expected value,
        # set its value to its intrinsic value
        intrin_value = cal_intrin_value(s, k, i)
        ind = cv[i - 1, :] < intrin_value
        v[i - 1, ind] = intrin_value[ind]
    # Return present values of the contract
    pv = np.exp(-r * h) * v[0, :]
    pv_red = (pv[:n] + pv[n:]) / 2

    return pv_red


In [6]:
# Output the results
berm_basket_price, berm_basket_var = reinforce_series_function(
    berm_basket_option, 100, 1e6)
print('Bermudan Basket Option:')
print('Price: {:.2f}'.format(berm_basket_price))
print('Variance of the Estimator: {:.4e}'.format(berm_basket_var))

Bermudan Basket Option:
Price: 20.15
Variance of the Estimator: 1.5966e-06


# Problem 2

In [7]:
def bs_formula_d1(st, k, r, sigma, t):
    """ Calculate the d1 value in the Black-Scholes model

    Parameters
    ----------
    st : float
        The spot price of the asset
    k : float
        The strike price
    r : float
        The risk-free rate
    sigma : float
        The volatility of the asset
    t : float
        The time to maturity

    Returns
    -------
    d1 : float
        The d1 value in the Black-Scholes model

    """

    d1 = (np.log(st / k) + (r + sigma ** 2 / 2) * t) / (sigma * np.sqrt(t))

    return d1


def bs_formula(st, k, r, sigma, t, type):
    """Use the Black-Scholes model to calculate the option's price

    Parameters
    ----------
    st : float
        The spot price of the asset
    k : float
        The strike price
    r : float
        The risk-free rate
    sigma : float
        The volatility of the asset
    t : float
        The time to maturity
    type : {'call', 'put'}
        The option type

    Returns
    -------
    v : float
        The option price

    """
    d1 = bs_formula_d1(st, k, r, sigma, t)
    d2 = d1 - sigma * np.sqrt(t)
    if type == 'call':
        v = norm.cdf(d1) * st - norm.cdf(d2) * k * np.exp(-r * t)
    elif type == 'put':
        v = norm.cdf(-d2) * k * np.exp(-r * t) - norm.cdf(-d1) * st

    return v


def d10_99var_delta_gamma_full(n):
    """Calculate the 10-day 99% VaR using the three methods

    Parameters
    ----------
    n : int or float
        The simulation size

    Returns
    -------
    d10_99var_delta : float
        The estimated 10-day 99% VaR using the delta method
    d10_99var_gamma : float
        The estimated 10-day 99% VaR using the delta-gamma method
    d10_99var_full : float
        The actual 10-day 99% VaR based on the simulation

    """

    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of options and assets
    sc0, kc, volc, mc = 50, 51, 0.28, 0.75
    sp0, kp, volp, mp = 20, 19, 0.25, 1
    r, rho, t = 0.06, 0.4, 10 / 252
    # Using the Black-Scholes formula, calculate d1 values for both options
    dc1 = bs_formula_d1(sc0, kc, r, volc, mc)
    dp1 = bs_formula_d1(sp0, kp, r, volp, mp)
    # Calculate both options' delta's and gamma's
    delta_c = norm.cdf(dc1)
    delta_p = norm.cdf(dp1) - 1
    gamma_c = norm.pdf(dc1) / (sc0 * volc * np.sqrt(mc))
    gamma_p = norm.pdf(dp1) / (sp0 * volp * np.sqrt(mp))
    # Data vectorization
    s0 = np.array([sc0, sp0]).reshape((2, -1))
    vol = np.array([volc, volp]).reshape((2, -1))
    delta = np.array([delta_c, delta_p]).reshape((2, -1))
    gamma = np.array([gamma_c, gamma_p]).reshape((2, -1))
    # Define parameters for the multivariate normal distribution
    mean = np.zeros(2)
    corr = np.full((2, 2), rho)
    np.fill_diagonal(corr, 1)
    # Generate multivariate normal random variables
    rng = np.random.default_rng()
    x = rng.multivariate_normal(mean, corr, size=n).T
    # Use the antithetic method to reduce the variance
    x = np.hstack((x, -x))
    # Calculate stock prices in 10 days
    st = s0 * (np.exp(
        (r - vol ** 2 / 2) * t + vol * np.sqrt(t) * x))
    # Use the delta method to estimate the 10-day 99% VaR
    ds = st - s0
    dv_delta = delta.T @ ds
    # Use the delta-gamma method to estimate
    dv_gamma = delta.T @ ds + gamma.T @ ds ** 2 / 2
    # Calculate fair prices of both options using the full simulation
    v = bs_formula(sc0, kc, r, volc, mc, 'call') + bs_formula(
        sp0, kp, r, volp, mp, 'put')
    vt = bs_formula(st[0, :], kc, r, volc, mc - t, 'call') + bs_formula(
        st[1, :], kp, r, volp, mp - t, 'put')
    dv = vt - v
    # Calculate the 10-day 99% VaR
    d10_99var_delta = np.quantile(dv_delta, .99)
    d10_99var_gamma = np.quantile(dv_gamma, .99)
    d10_99var_full = np.quantile(dv, .99)

    return d10_99var_delta, d10_99var_gamma, d10_99var_full


In [8]:
# Output the results
delta_es, gamma_es, full_es = reinforce_function(
    d10_99var_delta_gamma_full, 10, 1e7)
print('Estimated 10-day 99% VaR:')
print('Delta Method: {:.2f}'.format(delta_es))
print('Delta-Gamma Method: {:.2f}'.format(gamma_es))
print('Full Simulation Approach: {:.2f}'.format(full_es))

Estimated 10-day 99% VaR:
Delta Method: 3.87
Delta-Gamma Method: 4.70
Full Simulation Approach: 4.40


# Problem 3

In [9]:
def cal_default_prob(n):
    """

    Parameters
    ----------
    n : int or float
        The simulation size

    Returns
    -------
    default_prob : float
        The probability of default

    """

    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the asset and the contract
    v0, mu, sigma, b, t = 100, 0.03, 0.4, 70, 5
    # Generate normal random variables
    x = np.random.normal(size=n)
    # Use the antithetic method to reduce the variance
    x = np.hstack((x, -x))
    # Generate uniform random variables
    u = np.random.uniform(size=2 * n)
    # Using the property of the Brownian bridge, calculate minima
    inc_rate = (mu - sigma ** 2 / 2) * t + sigma * np.sqrt(t) * x
    m = np.exp((2 * np.log(v0) + inc_rate - np.sqrt(
        inc_rate ** 2 - 2 * sigma ** 2 * t * np.log(u))) / 2)
    # Derive the default probability
    default_prob = (m <= b).sum() / (2 * n)

    return default_prob


In [10]:
# Output the result
prob_d = reinforce_function(cal_default_prob, 10, 1e5)
print('Probability of Default: {:.2%}'.format(prob_d))

Probability of Default: 76.34%
